In [ ]:
import os

CUR_DIR = os.getcwd()
print("dir_name ", CUR_DIR)

# get the files from this directory

files_and_folders = os.listdir(CUR_DIR)
print("files: ", files_and_folders)


In [ ]:
folders_root = [f for f in files_and_folders if os.path.isdir(f)]
folders_root 

In [45]:
# traverse through every folder and get the verilog files
import re
from typing import Literal



class VerilogFile:
    @staticmethod
    def read_file_lines(file_name):
        with open(file_name, "r") as file:
            return file.readlines()

    def __get_by_name(self, name: Literal["input", "output"], file_name: str, seen:int=0) -> int:
        lines = VerilogFile.read_file_lines(file_name)
        inputs = []
        # print('File name = ', file_name)
        for i, line in enumerate(lines):
            if f" {name} " in line:  # find the first line with ;
                if seen > 0:
                    seen -= 1
                    continue
                for j in range(i, len(lines)):
                    cur_inputs = re.split(",| ", lines[j])  # split by comma or space
                    cur_inputs = [
                        i.replace(";", "").replace("'", "").replace(")", "").strip()
                        for i in cur_inputs
                        if i.strip()
                    ]
                    inputs.extend(cur_inputs)
                    # print("cur_inputs = ", cur_inputs)
                    if ";" in lines[j]:
                        return len(inputs) - 1
        raise ValueError("No inputs found in file")

    @staticmethod
    def get_number_inputs(file_name:str, seen:int=0 ) -> int:
        return VerilogFile().__get_by_name("input", file_name, seen) + seen 

    @staticmethod
    def get_number_outputs(file_name) -> int:
        return VerilogFile().__get_by_name("output", file_name)

    @staticmethod
    def get_io(file_name) -> int:
        return VerilogFile.get_number_inputs(
            file_name, seen=1
        ) + VerilogFile.get_number_outputs(file_name)


# VerilogFile.get_io(CUR_DIR + "\\" + folders_root[0] + "\\" + "c1355_encrypted.v")

In [ ]:
os.listdir(CUR_DIR + "/" + folders_root[0])

In [ ]:
# Get the IO of every file in every folder
import pandas as pd 
def get_io_folder(folder):
    files = os.listdir(CUR_DIR + "/" + folder)
    print('files = ', files)
    return [{'Benchmark_name': file, 'io' :VerilogFile.get_io(CUR_DIR + "/" + folder + "/" + file)} for file in files if file.endswith('.v')]

df = pd.DataFrame( get_io_folder(folders_root[0]))


for folder in folders_root:
    print('folder = ', folder)
    print(pd.DataFrame( get_io_folder(folder) ))
    print('\n\n')

In [17]:
from typing import Dict, List
from pprint import pprint
import pandas as pd

NAME_IN = 0 
LINE_IN = 1 
WORD_NUMBER = 2 
report_types :list[tuple[str, str, int]]= [  
                                        ('area', '<none> (D)', 4), 
                                        ('power', 'Subtotal', 4 )
                ] 

def get_files_in_folder(folder_name: str):
    folders = os.listdir(  folder_name)
    result : Dict[str, List[str]] = {"Benchmark_Name": [], "area": [], "power": []} 

    for f in folders: 
        dir_path = os.path.join(CUR_DIR, folder_name, f)
        # print(f'f={f} ')
        if not os.path.isdir(dir_path):
            continue
        for file in os.listdir(dir_path):
            for report, line_in, word_number in report_types:
                if report in file:
                    # print(f'\n\nf={f} ' )
                    # print(f'file = {file}')
                    if f not in result["Benchmark_Name"]:
                        result["Benchmark_Name"].append(f)
                    # check if the word is in the file, the get the value at the word number 
                    with open(os.path.join(dir_path, file), 'r') as f_:
                        lines = f_.readlines()
                        for line in lines:
                            if line_in in line:
                                # print('line = ', line)
                                words = line.split()
                                # print('words = ', words)
                                # print('value = ', words[word_number])
                                result[report].append(words[word_number])
    # pprint(result)
    return result


df = pd.DataFrame(get_files_in_folder(folders_root[0]))


In [ ]:
for folder in folders_root:
    print('  \n')
    print(f'folder = {folder}')
    df = pd.DataFrame(  get_files_in_folder(folder) ) 
    print(df)